In [1]:

%matplotlib inline
from __future__ import print_function

try:
    xrange
except NameError:
    xrange = range

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.spatial as sp
import scipy.sparse as sparse
from sklearn.decomposition import NMF, TruncatedSVD

In [2]:
# レコメンデーション
# ダミーデータを使ったサンプル
rating_matrix = np.array([[2, 5, 1, 1, 0, 1, 2, 4],
                          [1, 5, 2, 1, 4, 0, 0, 3],
                          [0, 3, 3, 0, 1, 1, 1, 1],
                          [5, 2, 2, 3, 1, 0, 0, 4],
                          [5, 3, 3, 4, 1, 0, 0, 5],
                          [1, 4, 3, 2, 5, 1, 0, 1],
                          [0, 0, 0, 0, 0, 0, 0, 2],
                          [0, 4, 0, 0, 0, 0, 0, 0]])

In [3]:
# 1行目のユーザーとにているユーザーを探す
user_similarity = []
for i in range(len(rating_matrix)):
    # cosine類似度　= 1 - cosine距離
    sim = 1 - sp.distance.cosine(rating_matrix[0], rating_matrix[i])
    user_similarity.append(sim)
    
user_similarity = np.array(user_similarity)
user_similarity

array([ 1.        ,  0.77831178,  0.73914049,  0.7402121 ,  0.78215389,
        0.58777469,  0.5547002 ,  0.69337525])

In [4]:
# 類似度が高いユーザーを探す
topN = 4

# userのインデックス（行番号）をNumpy Arrayで作っておきます
user_idx = np.array(range(0, len(rating_matrix)))

# 類似度の低い順にソートした結果のインデックスを用意して、降順に並び替えます
arg_sort = np.argsort(user_similarity)
arg_sort = arg_sort[::-1]

# 自分自身を除いた類似度の高い4人を選びます
selected_idx = arg_sort[1:topN+1]

selected_user_similarity = user_similarity[selected_idx]
selected_rating_matrix = rating_matrix[selected_idx]
selected_rating_matrix

array([[5, 3, 3, 4, 1, 0, 0, 5],
       [1, 5, 2, 1, 4, 0, 0, 3],
       [5, 2, 2, 3, 1, 0, 0, 4],
       [0, 3, 3, 0, 1, 1, 1, 1]])

In [5]:
selected_user_similarity

array([ 0.78215389,  0.77831178,  0.7402121 ,  0.73914049])

In [6]:
# 平均類似度
avg_score = []
for col_idx in range(selected_rating_matrix.shape[1]):
    weight_score = \
     sum(selected_rating_matrix[:, col_idx] * selected_user_similarity)
    similarity_sum = \
     sum(selected_user_similarity[selected_user_similarity > 0])
    avg_score.append(weight_score / similarity_sum)

In [7]:
for i, v in enumerate(avg_score):
    print(str(i) + 'th item  ', v)

0th item   2.76008004028
1th item   3.26857245915
2th item   2.50045570209
3th item   2.01576643714
4th item   1.76811675706
5th item   0.243152856729
6th item   0.243152856729
7th item   3.27180535615


In [8]:
# 某ニュースアプリのテーマのフォロー状況を模したデータで協調フィルタリング
data = pd.read_csv('dataset/user_topic_follow_dummy.csv', encoding='utf8')
# DataFrame.drop_duplicates。重複のうち、最初のひとつ もしくは 最後のひとつ どちらかを残す
data.drop_duplicates(keep='last', inplace=True)
print(data.shape)
data.head()

(339323, 2)


,user_id,topic_name
0,25126,(株)アップル
1,26285,(株)電通
2,15409,.NET_Framework
3,30466,.NET_Framework
4,30878,.NET_Framework


In [9]:
data['rating'] = 1.0

In [10]:
# 複数行持ちの値を列持ちに変換 (pivot)
# DataFrame.pivot。集約処理付きの別関数 pd.pivot_table もある。
rating_matrix = \
 data.pivot(index='user_id', columns='topic_name', values='rating')
rating_matrix.fillna(0, inplace=True)


In [11]:
topic_list = np.array(rating_matrix.columns)
user_list = np.array(rating_matrix.index)

In [12]:
rating_matrix_ar = np.array(rating_matrix)
rating_matrix_ar

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [13]:
# さて、user_id=1の方に対するオススメトピックを探してみましょう。
already_followed_topic = \
 np.array(data[data['user_id'] == user_list[0]]['topic_name'])
data[data['user_id'] == user_list[0]]

,user_id,topic_name,rating
40826,1,MacBook,1.0
187716,1,一人旅,1.0
211960,1,吉祥寺,1.0
287441,1,築地市場,1.0
327071,1,離乳食,1.0


In [14]:
# 類似度計算
def get_cosine_similarity(x, y):
    return 1- sp.distance.cosine(x, y)

def get_jaccard_similarity(x, y):
    return 1 - sp.distance.jaccard(x, y)

user_similarity = []
target_user_row = rating_matrix_ar[0]
for row in rating_matrix_ar:
    sim = get_jaccard_similarity(target_user_row, row)
    user_similarity.append(sim)
user_similarity = np.array(user_similarity)

In [15]:
user_similarity

array([ 1.        ,  0.        ,  0.14285714, ...,  0.        ,
        0.08333333,  0.05263158])

In [16]:
# 類似度が高いユーザー
topN = 20
idx = user_similarity.argsort()[::-1][1:topN+1]
selected_user_similarity = user_similarity[idx]
selected_rating = rating_matrix_ar[idx]

In [17]:
# 平均類似度計算
avg_score = []
for col_idx in range(selected_rating.shape[1]):
    weight_score = sum(selected_rating[:, col_idx] * selected_user_similarity)
    similarity_sum = \
     sum(selected_user_similarity[selected_user_similarity > 0])
    avg_score.append(weight_score / similarity_sum)
avg_score = np.array(avg_score)

In [18]:
recommend_num = 5
counter = 0
for recommended_topic in topic_list[avg_score.argsort()[::-1]]:
    if recommended_topic not in already_followed_topic:
        print(recommended_topic)
        counter += 1
        if recommend_num <= counter:
            break

インテリジェント・デザイン
映画
音楽
デート
副業


In [19]:
# 次元削減
# 特異値分解(Singular Value Decomposition)による次元圧縮を使ったレコメンデーション
svd = TruncatedSVD(n_components=20)

In [20]:
rating_matrix_sparse = sparse.lil_matrix(rating_matrix_ar)

In [21]:
rating_matrix_svd = svd.fit_transform(rating_matrix_sparse)

In [22]:
user_similarity = []
target_user_svd = rating_matrix_svd[0]
for row in rating_matrix_svd:
    sim = get_cosine_similarity(target_user_svd, row)
    user_similarity.append(sim)
user_similarity = np.array(user_similarity)

In [23]:
topN = 20
idx = user_similarity.argsort()[::-1][1:topN+1]
selected_user_similarity = user_similarity[idx]
selected_rating = rating_matrix_ar[idx]

avg_score = []
for col_idx in range(selected_rating.shape[1]):
    weight_score = sum(selected_rating[:, col_idx] * selected_user_similarity)
    similarity_sum = \
     sum(selected_user_similarity[selected_user_similarity > 0])
    avg_score.append(weight_score / similarity_sum)
avg_score = np.array(avg_score)

recommend_num = 5
counter = 0
for recommended_topic in topic_list[avg_score.argsort()[::-1]]:
    if recommended_topic not in already_followed_topic:
        print(recommended_topic)
        counter += 1
        if recommend_num <= counter:
            break

インテリジェント・デザイン
ロードバイク
Twitter
ゲーム
PlayStation_4


In [24]:
# 非負値行列分解(Non Negative Matrix Factorization)による次元圧縮を使ったレコメンデーション
nmf = NMF(n_components=10, random_state=1234, init='random')

In [25]:
rating_matrix_nmf = nmf.fit_transform(rating_matrix_sparse)

In [26]:
user_similarity = []
target_user_nmf = rating_matrix_nmf[0]
for row in rating_matrix_nmf:
    if sum(row) == 0:
        sim = 0
        user_similarity.append(sim)
    else:
        sim = get_cosine_similarity(target_user_nmf, row)
        user_similarity.append(sim)
user_similarity = np.array(user_similarity)

In [27]:
topN = 20
idx = user_similarity.argsort()[::-1][1:topN+1]
selected_user_similarity = user_similarity[idx]
selected_rating = rating_matrix_ar[idx]

avg_score = []
for col_idx in range(selected_rating.shape[1]):
    weight_score = sum(selected_rating[:, col_idx] * selected_user_similarity)
    similarity_sum = \
     sum(selected_user_similarity[selected_user_similarity > 0])
    avg_score.append(weight_score / similarity_sum)
avg_score = np.array(avg_score)

recommend_num = 5
counter = 0
for recommended_topic in topic_list[avg_score.argsort()[::-1]]:
    if recommended_topic not in already_followed_topic:
        print(recommended_topic)
        counter += 1
        if recommend_num <= counter:
            break

転職
でんぱ組.inc
インテリジェント・デザイン
クラフトビール
Twitter
